# Computing Shapley values for a fully connected 2 layers net 

In [34]:
import torch
import torch.nn.functional as F
from math import factorial
from itertools import chain, combinations


In [3]:
x = torch.tensor([1.0,2.0,3.0])
x_p = torch.tensor([1.0,-2.0,3.0])

In [10]:
class Net(torch.nn.Module):
    def forward(self,x):
        B = torch.tensor([[1,1,0],[0,1,1]], dtype=torch.float)
        A = torch.tensor([[3,0,0],[0,-2,0],[0,0,1]], dtype=torch.float)
        return B@F.relu(A@x)

In [13]:
net = Net()
net(x) #testing

tensor([3., 3.])

In [55]:
def get_shapley(x, net):
    """ Computes attribution for all features and all 
    outputs of input x """
    input_size = x.size()[0]
    shapleys = [0]*input_size
    P = set(range(input_size))
    for i in range(input_size):
        # i = 0,1,2
        # Compute attributions to feature i 
        input_features_no_i = list(range(i)) + list(range(i+1,input_size))
        all_subs = chain.from_iterable(combinations(input_features_no_i, r) for r in range(input_size))
        for s in all_subs: # all feature subsets 
            S = set(s)
            factor = (factorial(len(S))*factorial(input_size-1-len(S)))/factorial(input_size)
            input_with = x.clone()
            input_without = x.clone()
            for j in P-S-{i}: input_with[j] = 0
            for j in P-S: input_without[j] = 0
            out_with = net(input_with)
            out_without = net(input_without)
            shapleys[i] += factor*(out_with-out_without)

    return shapleys

In [56]:
get_shapley(x, net)

[tensor([3., 0.]), tensor([0., 0.]), tensor([0., 3.])]

In [57]:
get_shapley(x_p, net)

[tensor([3., 0.]), tensor([4., 4.]), tensor([0., 3.])]